In [ ]:
# Building Diverse Paraphraser

In [1]:
import torch
import tqdm

In [2]:
# declaring variables
model_type = "gpt2"
model_name_or_path = "gpt2" #gpt2-large
data_dir = ""
save_steps = 500
num_epochs = 3
gradient_accumulation_steps = 2
per_gpu_train_batch_size = 5
learning_rt = 5e-5
prefix_input_type = "original"
extra_embedding_dim = 768 # Size of linear layer used for projecting extra embeddings.
global_dense_feature_list = None

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

print(device, n_gpus)

cpu 0


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00


In [5]:
from transformers import (AdamW, GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, get_linear_schedule_with_warmup)

In [6]:
MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
}

In [7]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
print(config_class, model_class, tokenizer_class, sep="\n")

<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
<class 'transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer'>


In [8]:
config = config_class.from_pretrained(model_name_or_path)

In [9]:
config.extra_embedding_dim = extra_embedding_dim # don't know why we are using it; -- explore it

In [10]:
tokenizer = tokenizer_class.from_pretrained(model_name_or_path,do_lower_case = False)

In [11]:
model = model_class.from_pretrained(model_name_or_path, config=config)

In [12]:
SPECIAL_TOKENS = {
    "additional_special_tokens": ["<dense-vectors>", "<tokens>", "<vercourb>", "<ARG0>", "<ARG1>", "<global-dense-vectors>"],
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
} # total 9 tokens

In [13]:
tokenizer.add_special_tokens(SPECIAL_TOKENS)

9

In [14]:
len(tokenizer) # in general 50257; but 9 are added so becomes 50266

50266

In [15]:
model.resize_token_embeddings(len(tokenizer)) # each token of size-> 1280

Embedding(50266, 768)

In [16]:
# move model to gpu
# model.to(device)

In [16]:
import tqdm
from torch.utils.data import Dataset
import pickle
import numpy as np

In [17]:
from collections import defaultdict

BATCH_SIZE = 128
MAX_LENGTH = 502
MAX_GPT2_LENGTH = 1000

MAX_PARAPHRASE_LENGTH = 10

BASE_CONFIG = {
    "keys": [
        {"key": "sent1_tokens", "position": 3, "tokenize": True, "metadata": False},
        {"key": "sent2_tokens", "position": 4, "tokenize": True, "metadata": False},
        {"key": "f1_score", "position": 5, "tokenize": False, "metadata": True},
        {"key": "kt_score", "position": 6, "tokenize": False, "metadata": True},
        {"key": "ed_score", "position": 7, "tokenize": False, "metadata": True},
        {"key": "langid", "position": 8, "tokenize": False, "metadata": True},
    ],
    "max_total_length": MAX_PARAPHRASE_LENGTH,
    "max_prefix_length": int(MAX_PARAPHRASE_LENGTH / 2),
    "max_suffix_length": int(MAX_PARAPHRASE_LENGTH / 2),
    "max_dense_length": 2
}

In [18]:
DATASET_CONFIG = {
"datasets/test_custom": BASE_CONFIG,
"datasets/paranmt_filtered": BASE_CONFIG,
    "datasets/shakespeare": BASE_CONFIG,
    "datasets/formality": BASE_CONFIG,
    "datasets/shakespeare_aae_tweets_bible_romantic-poetry_switchboard_coha_3_bins_lyrics_full": BASE_CONFIG,
    "datasets/aae": BASE_CONFIG,
    "datasets/bible": BASE_CONFIG,
    "datasets/poetry": BASE_CONFIG,
    "datasets/switchboard": BASE_CONFIG,
    "datasets/english_tweets": BASE_CONFIG,
    "datasets/lyrics_full": BASE_CONFIG,
    "datasets/joyce": BASE_CONFIG,
    "datasets/congress-bills": BASE_CONFIG,
    "datasets/shakespeare": BASE_CONFIG,
    "datasets/coha_3_bins_1810s-1820s": BASE_CONFIG,
    "datasets/coha_3_bins_1890s-1900s": BASE_CONFIG,
    "datasets/coha_3_bins_1990s-2000s": BASE_CONFIG
}

In [19]:
def string_to_ids(text, tokenizer):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [20]:
tokenizer.pad_token_id

50263

In [21]:
def datum_to_dict(config, datum, tokenizer):
    instance_dict = {"metadata": ""}

    for key in config["keys"]:
        element_value = datum[key["position"]]
        instance_dict[key["key"]] = string_to_ids(element_value, tokenizer) if key["tokenize"] else element_value
        if key["metadata"]:
            instance_dict["metadata"] += "%s = %s, " % (key["key"], str(element_value))

    # print("Instance dict metadata:- ", instance_dict["metadata"])
    # strip off trailing , from metadata
    instance_dict["metadata"] = instance_dict["metadata"][:-2] # just to remove last space and comma
    # print("Instance dict metadata after trailing off :- ", instance_dict["metadata"])

    return instance_dict

In [22]:
limit_examples = 10

In [1]:
def preprocess(exp, tokenizer, do_tokenize=True):
  # truncate
  max_prefix_len = BASE_CONFIG["max_prefix_length"]
  max_suffix_len = BASE_CONFIG["max_suffix_length"]

  sent1 = np.array(exp["sent1_tokens"])
  sent2 = np.array(exp["sent2_tokens"])

  if(len(sent1) > max_prefix_len):
    sent1 = sent1[:max_prefix_len]

  if(len(sent2) > max_suffix_len):
    sent2 = sent2[:max_suffix_len]

  # shuffle (prefix and suffix; we will not do this)

  # add padding; left padding to prefix and right padding to suffix
  count_pad_tokens_prefix = max_prefix_len - len(sent1)
  sent1 = np.pad(sent1, (count_pad_tokens_prefix, 0), constant_values = tokenizer.pad_token_id)

  # add <eos> to suffix
  sent2 = np.append(sent2, tokenizer.eos_token_id)

  count_pad_tokens_suffix = (max_suffix_len + 1) - len(sent2)
  sent2 = np.pad(sent2, (0, count_pad_tokens_suffix), constant_values = tokenizer.pad_token_id)

  # sentence to input gpt2
  sentence_to_input_gpt2 = np.concatenate([sent1, [tokenizer.bos_token_id], sent2]).astype(np.int64) # [sent1, <bos> sent2]

  # label/gt to predict
  gt = np.concatenate([
      [-100 for _ in sent1],
      [-100],
      [val if val != tokenizer.pad_token_id else -100 for val in sent2]
  ]).astype(np.int64)

  # segment
  segment = np.concatenate([
      [tokenizer.additional_special_tokens_ids[1] for _ in sent1],
      [tokenizer.additional_special_tokens_ids[2]],
      [tokenizer.additional_special_tokens_ids[2] for _ in sent2]
  ]).astype(np.int64)

  exp["prefix_sent"] = sent1
  exp["suffix_sent"] = sent2

  exp["input"] = sentence_to_input_gpt2
  exp["label"] = gt
  exp["segment"] = segment

  return exp

In [24]:
# custom dataset
class ParaphraseDatasetText(Dataset):
    def __init__(self, tokenizer, evaluate=False, split="train"):
        data_dir = "/content/drive/MyDrive/IRE_Project/style_transfer_paraphrase/datasets/paranmt_filtered/"+split+".pickle"
        # self.args = args

        if data_dir in DATASET_CONFIG:
            self.config = DATASET_CONFIG[data_dir]
        else:
            self.config = BASE_CONFIG

        # update_config(self.args, self.config)

        self.examples = []

        # Caching is important since it can avoid slow tokenization
        with open(data_dir, "rb") as handle:
          parse_data = pickle.load(handle)

        print("Original raw data- ", parse_data[0])

        self.examples = [datum_to_dict(self.config, datum, tokenizer) for datum in tqdm.tqdm(parse_data)]

        print("\n------------------ After conversion- ", self.examples[0])

        # in case we are using a fraction of the dataset, reduce the size of the dataset here
        self.examples = self.examples[:limit_examples]

        # convert the dataset into the Instance class
        # self.examples = [Instance(args, self.config, datum_dict) for datum_dict in self.examples]
        self.examples = [preprocess(exp, tokenizer) for exp in self.examples]
        print(self.examples[0])
        # print("Metadata", self.examples[0].dict["metadata"])

        # num_truncated = sum([x.truncated for x in self.examples]) # to get truncated count in examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        sentence = self.examples[item]["input"]
        label = self.examples[item]["label"]
        segment = self.examples[item]["segment"]
        suffix_style = 0
        original_style = 0 # since in paraphrase, both will remain in the same style
        init_context_size = 1 + BASE_CONFIG["max_prefix_length"]

        global_dense_vectors = np.zeros((1, 768), dtype=np.float32)

        return {
            "sentence": torch.tensor(sentence),
            "instance_number": item,
            "label": torch.tensor(label),
            "segment": torch.tensor(segment),
            "suffix_style": suffix_style,
            "original_style": original_style,
            "init_context_size": init_context_size,
            "global_dense_vectors": global_dense_vectors,
            # "metadata": self.examples[item].dict["metadata"],
        }

In [25]:
train_dataset = ParaphraseDatasetText(tokenizer=tokenizer)

Original raw data-  ('S > NP VP .', 'S > SBAR , NP VP .', False, 'Mr. Whetstone is goingto speak to you after I finish.', "after I'm done, Mr. Whetstone will be speaking.", (0.3, 0.375, 0.33333333333333326), (-0.33333333333333337, False), 2, ('en', 'en'))


100%|██████████| 73062/73062 [00:30<00:00, 2432.01it/s]



------------------ After conversion-  {'metadata': "f1_score = (0.3, 0.375, 0.33333333333333326), kt_score = (-0.33333333333333337, False), ed_score = 2, langid = ('en', 'en')", 'sent1_tokens': [5246, 13, 370, 3202, 6440, 318, 1016, 1462, 2740, 284, 345, 706, 314, 5461, 13], 'sent2_tokens': [8499, 314, 1101, 1760, 11, 1770, 13, 370, 3202, 6440, 481, 307, 5486, 13], 'f1_score': (0.3, 0.375, 0.33333333333333326), 'kt_score': (-0.33333333333333337, False), 'ed_score': 2, 'langid': ('en', 'en')}
{'metadata': "f1_score = (0.3, 0.375, 0.33333333333333326), kt_score = (-0.33333333333333337, False), ed_score = 2, langid = ('en', 'en')", 'sent1_tokens': [5246, 13, 370, 3202, 6440, 318, 1016, 1462, 2740, 284, 345, 706, 314, 5461, 13], 'sent2_tokens': [8499, 314, 1101, 1760, 11, 1770, 13, 370, 3202, 6440, 481, 307, 5486, 13], 'f1_score': (0.3, 0.375, 0.33333333333333326), 'kt_score': (-0.33333333333333337, False), 'ed_score': 2, 'langid': ('en', 'en'), 'prefix_sent': array([5246,   13,  370, 320

In [26]:
next(iter(train_dataset))

{'sentence': tensor([ 5246,    13,   370,  3202,  6440, 50264,  8499,   314,  1101,  1760,
            11, 50265]),
 'instance_number': 0,
 'label': tensor([ -100,  -100,  -100,  -100,  -100,  -100,  8499,   314,  1101,  1760,
            11, 50265]),
 'segment': tensor([50258, 50258, 50258, 50258, 50258, 50259, 50259, 50259, 50259, 50259,
         50259, 50259]),
 'suffix_style': 0,
 'original_style': 0,
 'init_context_size': 6,
 'global_dense_vectors': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange

In [28]:
# start training
train_batch_size = per_gpu_train_batch_size
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=train_batch_size)

In [29]:
len(train_dataloader)  # basically how many batches will be formed

2

In [30]:
per_gpu_train_batch_size

5

In [31]:
# total gradient descent steps
t_total = len(train_dataloader) // gradient_accumulation_steps * num_epochs
t_total

3

In [32]:
adam_epsilon = 1e-8
warmup_steps = 0

In [33]:
# setting up optimizer and scheduler for training
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']

grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters()],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(grouped_parameters,
                lr=float(learning_rt),
                eps=adam_epsilon)

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [34]:
# total information before training
print("Num of examples- ", len(train_dataset))
print("Num of epochs- ", num_epochs)
print("Batch size- ", train_batch_size)
print("Gradient acculmulation steps- ", gradient_accumulation_steps)
print("Total optimization steps- ", t_total)

Num of examples-  10
Num of epochs-  3
Batch size-  5
Gradient acculmulation steps-  2
Total optimization steps-  3


In [35]:
model.zero_grad()

In [36]:
train_iterator = trange(int(num_epochs), desc="Epoch")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
for epoch in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
    for batch_idx, batch in enumerate(epoch_iterator):
      # print(batch_idx)
      print((batch))
      break
      # loss = model(batch)


Iteration:   0%|          | 0/2 [00:00<?, ?it/s]


{'sentence': tensor([[   40,  1101,  1342,   621,  1315, 50264,    40,  1183,   307,   351,
          6550, 50265],
        [   40,  4724,   340,   338,   510, 50264,   568,  8914,   616,  1641,
           318, 50265],
        [ 5246,    13,   370,  3202,  6440, 50264,  8499,   314,  1101,  1760,
            11, 50265],
        [ 1026,  1183,  2866,  6082,   618, 50264,   732,   561, 13604,   578,
           884, 50265],
        [41481,   399,  1268,  2767,    36, 50264, 12518,  1709,   504, 49325,
           338, 50265]]), 'instance_number': tensor([3, 1, 0, 2, 7]), 'label': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,    40,  1183,   307,   351,
          6550, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,   568,  8914,   616,  1641,
           318, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,  8499,   314,  1101,  1760,
            11, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,   732,   561, 13604,   578,
           884, 50265],
     


Iteration:   0%|          | 0/2 [00:00<?, ?it/s]


{'sentence': tensor([[ 1639,  1053,  1760,   257,  1049, 50264,  5832,  4376,   674,  4957,
          1049, 50265],
        [ 1532,   340,  1549,  2152,    11, 50264,   361,   339,  1107, 11196,
            11, 50265],
        [ 1026,   743,  2291,    11,   379, 50264, 38169,   284,   262,  3572,
            11, 50265],
        [ 1026,  1183,  2866,  6082,   618, 50264,   732,   561, 13604,   578,
           884, 50265],
        [ 5246,    13,   370,  3202,  6440, 50264,  8499,   314,  1101,  1760,
            11, 50265]]), 'instance_number': tensor([8, 4, 6, 2, 0]), 'label': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  5832,  4376,   674,  4957,
          1049, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,   361,   339,  1107, 11196,
            11, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100, 38169,   284,   262,  3572,
            11, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,   732,   561, 13604,   578,
           884, 50265],
     


Epoch: 100%|██████████| 3/3 [00:00<00:00, 19.87it/s]

{'sentence': tensor([[18380,  1310,   645,   310, 35735, 50264,  1169, 10296,  7838,   286,
           262, 50265],
        [   40,  1101,  1342,   621,  1315, 50264,    40,  1183,   307,   351,
          6550, 50265],
        [ 1026,   743,  2291,    11,   379, 50264, 38169,   284,   262,  3572,
            11, 50265],
        [    1,  2484,   439,   484,   307, 50264,     1,  4598,   345,  1577,
           606, 50265],
        [ 5246,    13,   370,  3202,  6440, 50264,  8499,   314,  1101,  1760,
            11, 50265]]), 'instance_number': tensor([9, 3, 6, 5, 0]), 'label': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  1169, 10296,  7838,   286,
           262, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,    40,  1183,   307,   351,
          6550, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100, 38169,   284,   262,  3572,
            11, 50265],
        [ -100,  -100,  -100,  -100,  -100,  -100,     1,  4598,   345,  1577,
           606, 50265],
     

In [38]:
batch["sentence"].shape

torch.Size([5, 12])

In [39]:
max_grad_norm = 1.0
save_steps = 2
global_step = 0
loss_val = 0.0

In [40]:
import os

In [41]:
def save_model(model, tokenizer, chkpt_dir):
  if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir)

  print("Directory created")
  model.save_pretrained(chkpt_dir)
  tokenizer.save_pretrained(chkpt_dir)

  # save training arguments also

  with open(os.path.join(chkpt_dir, "global_step.txt"), "w") as f:
    f.write(str(global_step) + "\n")

  print("Model saved")

In [42]:
for epoch in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for batch_idx, batch in enumerate(epoch_iterator):
      sentences = batch["sentence"].to(device)
      labels = batch["label"].to(device)
      segments = batch["segment"].to(device)
      global_dense_vectors = batch["global_dense_vectors"].to(device)

      # print(sentences.shape)
      model.train()

      outputs = model(
                input_ids=sentences,
                token_type_ids=segments,
                labels=labels
            )

      loss = outputs.loss
      loss = loss / gradient_accumulation_steps
      loss_val += loss.item()

      # chkpt_dir = "/content/drive/MyDrive/IRE/chkkpt_"+str(global_step)
      # save_model(model, tokenizer, chkpt_dir)

      loss.backward()

      if (((batch_idx + 1) % gradient_accumulation_steps) == 0):
        # do gradient descent after gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        model.zero_grad()
        global_step += 1

        if (global_step % save_steps == 0):
          # save model here
          chkpt_dir = "/content/drive/MyDrive/IRE/chkpt_"+str(global_step)
          save_model(model, tokenizer, chkpt_dir)

      break

Iteration:   0%|          | 0/2 [00:06<?, ?it/s]


In [43]:
# loss_val
# avg_loss_per_step = loss_val / global_step
global_step, tr_loss = global_step, loss_val / (global_step+1) # +1 only while testing

In [63]:
global_step = 3

In [42]:
# save the last state also
chkpt_dir = "/content/drive/MyDrive/IRE/chkpt_"+str(global_step)
save_model(model, tokenizer, chkpt_dir)

Directory created
Model saved


In [43]:
# load from the checkpoints for evaluation
chkpt_to_load = chkpt_dir
model = model_class.from_pretrained(chkpt_to_load)
model.to(device)

tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
# do evaluation


In [44]:
from transformers import WEIGHTS_NAME
from tqdm import  tqdm
import glob

In [45]:
chkpts = ["chkpt_3"]

In [46]:
print(chkpts)

['chkpt_3']


In [47]:
perplexity_list = []

In [50]:
import tqdm

In [51]:
# create val dataloader; same way as in train dataloader
val_dataset = ParaphraseDatasetText(tokenizer=tokenizer, evaluate=True, split="dev")

Original raw data-  ('FRAG > CD , CD , CD , CD , CC VP .', 'FRAG > CD , CD , CD , CD CC VBP .', False, '1, 2, 3, 4, and get the hell out of there!', 'one, two, three, four and fall!', (0.1, 0.16666666666666666, 0.125), (-1.0, True), 2, ('en', 'en'))


100%|██████████| 1492/1492 [00:01<00:00, 1288.72it/s]


------------------ After conversion-  {'metadata': "f1_score = (0.1, 0.16666666666666666, 0.125), kt_score = (-1.0, True), ed_score = 2, langid = ('en', 'en')", 'sent1_tokens': [16, 11, 362, 11, 513, 11, 604, 11, 290, 651, 262, 5968, 503, 286, 612, 0], 'sent2_tokens': [505, 11, 734, 11, 1115, 11, 1440, 290, 2121, 0], 'f1_score': (0.1, 0.16666666666666666, 0.125), 'kt_score': (-1.0, True), 'ed_score': 2, 'langid': ('en', 'en')}
{'metadata': "f1_score = (0.1, 0.16666666666666666, 0.125), kt_score = (-1.0, True), ed_score = 2, langid = ('en', 'en')", 'sent1_tokens': [16, 11, 362, 11, 513, 11, 604, 11, 290, 651, 262, 5968, 503, 286, 612, 0], 'sent2_tokens': [505, 11, 734, 11, 1115, 11, 1440, 290, 2121, 0], 'f1_score': (0.1, 0.16666666666666666, 0.125), 'kt_score': (-1.0, True), 'ed_score': 2, 'langid': ('en', 'en'), 'prefix_sent': array([ 16,  11, 362,  11, 513]), 'suffix_sent': array([  505,    11,   734,    11,  1115, 50265]), 'input': array([   16,    11,   362,    11,   513, 50264,   

In [52]:
per_gpu_eval_batch_size = 2
eval_batch_size = per_gpu_eval_batch_size

In [55]:
# create val dataloader
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=eval_batch_size)

In [60]:
next(iter(val_dataloader))

{'sentence': tensor([[   16,    11,   362,    11,   513, 50264,   505,    11,   734,    11,
           1115, 50265],
         [   32,  1048,   338,  2095,   318, 50264,   259,   262,   266,  6696,
             11, 50265]]),
 'instance_number': tensor([0, 1]),
 'label': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,   505,    11,   734,    11,
           1115, 50265],
         [ -100,  -100,  -100,  -100,  -100,  -100,   259,   262,   266,  6696,
             11, 50265]]),
 'segment': tensor([[50258, 50258, 50258, 50258, 50258, 50259, 50259, 50259, 50259, 50259,
          50259, 50259],
         [50258, 50258, 50258, 50258, 50258, 50259, 50259, 50259, 50259, 50259,
          50259, 50259]]),
 'suffix_style': tensor([0, 0]),
 'original_style': tensor([0, 0]),
 'init_context_size': tensor([6, 6]),
 'global_dense_vectors': tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]]])}

In [57]:
# validation info
print("Num of examples- ", len(val_dataset))
print("Batch size- ", eval_batch_size)

Num of examples-  10
Batch size-  2


In [71]:
def evaluate(model, tokenizer, chkpt_name, val_dataloader):
  perplexity = 1.0
  val_loss = 0.0
  model.eval()

  for i, batch in enumerate(val_dataloader):
    sentences = batch["sentence"].to(device)
    labels = batch["label"].to(device)
    segments = batch["segment"].to(device)
    global_dense_vectors = batch["global_dense_vectors"].to(device)

    with torch.no_grad():
      op = model(
          input_ids=sentences,
          token_type_ids=segments,
          labels=labels
      )

      loss_val = op.loss.item()

    val_loss += loss_val
    break

  avg_val_loss = val_loss / (i + 1) # per batch average loss
  perplexity = torch.exp(torch.tensor(avg_val_loss))

  return perplexity

In [75]:
perplexity_list = []
for chkpt in chkpts:
  # load chkpoint
  chkpt_to_load = "/content/drive/MyDrive/IRE/" + chkpt
  model = model_class.from_pretrained(chkpt_to_load)
  model.to(device)

  # no need to load tokenizer again and again; as it remains same across all the chkpoints
  perplexity = evaluate(model, tokenizer, chkpt_dir, val_dataloader)
  perplexity_list.append((chkpt, perplexity))

# get sorted list
perplexity_list.sort(key=lambda x: x[1].item())
top_chkpt = perplexity_list[0][0]

# we can also delete the save points that are not needed
# can also move best checkpt to some other directory also for segregation

In [76]:
perplexity_list

[('chkpt_3', tensor(1.6906e+28))]

In [ ]:
# done evaluation part also

In [ ]:
# create paraphrased model

# 1. load the best DVP model

# 2. Convert shakespeare sentences to their paraphrased version

# 3. Convert sentences to their bpe files

In [ ]:
# 1. load the best DVP model and its tokenizer
chkpt_to_load = "/content/drive/MyDrive/IRE/chkpt_3/"
paraphraser = model_class.from_pretrained(chkpt_to_load)
paraphraser.to(device)

tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)

# load arguments also from file
config = BASE_CONFIG

# init some more variables for generation
upper_length = "same_5" # defautl is same_10
beam_size = 1 #default
num_samples = 1
temparature = 0
top_p = 0.0
device = device
top_k = 1
batch_size = 64

In [1]:
def get_logits(model, iteration, generated, segments, style_content_vectors, past):
    if iteration == 0:
        if style_content_vectors is None:
            pred = model(
                input_ids=generated,
                token_type_ids=segments,
                return_dict=True
            )
        else:
            pred = model(
                input_ids=generated,
                token_type_ids=segments,
                prefix_input_vectors=style_content_vectors,
                return_dict=True
            )
    else:
        # used the cached representations to speed up decoding
        pred = model(
            input_ids=generated[:, -1:],
            token_type_ids=segments[:, -1:],
            past_key_values=past,
            return_dict=True
        )
    logits = pred['logits']
    past = pred['past_key_values']
    return logits, past

In [ ]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
  top_k = min(top_k, logits.size(-1))  # Safety check
  if top_p > 0.0:
      sorted_logits, sorted_indices = torch.sort(logits, descending=True)
      cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
      # Remove tokens with cumulative probability above the threshold
      sorted_indices_to_remove = cumulative_probs > top_p
      # Shift the indices to the right to keep also the first token above the threshold
      sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
      sorted_indices_to_remove[..., 0] = 0

      # scatter sorted tensors to original indexing
      indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
      logits[indices_to_remove] = filter_value

  elif top_k > 0:
      # Remove all tokens with a probability less than the last token of the top-k
      indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
      logits[indices_to_remove] = filter_value

  return logits

In [ ]:
def sample_sequence(model, length, context, style_content_vectors, segments, eos_token_id,
                    num_samples=1, temperature=1, top_k=0, top_p=0.0, get_scores=False,
                    interpolation=None):
  new_gen_length = length
  batch_size = context.shape[0] # total sents
  eos_emitted = [False for _ in range(batch_size)]

  generated = context
  scores = [{"score": 0, "sequence": []} for _ in range(batch_size)]

  with torch.no_grad():
    past = None

    for i in range(new_gen_length):

      logits, past = get_logits(model, i, generated, segments, style_content_vectors, past)
      next_token_logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.)
      original_scores = F.log_softmax(next_token_logits, dim=-1)

      filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)

      if temperature == 0 and top_k in [0, 1] and top_p == 0.0:
        # greedy sampling
        next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)

      if get_scores:
        for batch_elem in range(batch_size):
            if eos_emitted[batch_elem]:
                continue
            scores[batch_elem]["score"] += original_scores[batch_elem, next_token[batch_elem].item()].item()
            scores[batch_elem]["sequence"].append("token")

      generated = torch.cat((generated, next_token), dim=1)
      segments = torch.cat((segments, segments[:, -1:]), dim=1)

      for batch_elem in range(batch_size):
        if next_token[batch_elem].item() == eos_token_id:
            eos_emitted[batch_elem] = True

      if length is None and all(eos_emitted):
        break

  if get_scores:
    scores = [x["score"] / len(x["sequence"]) for x in scores]

  return generated, scores

In [ ]:
def generate(paraphraser, gpt2_sentences, segments, global_dense_vectors=None,
                 init_context_size=1, eos_token_id=None, get_scores=False,
                 interpolation=None, top_p=None):

  style_content_vectors = None
  generation_length = len(gpt2_sentences[0]) - init_context_size
  dense_length = 0

  out, scores = sample_sequence(
                model = paraphraser,
                context = gpt2_sentences[:, 0:init_context_size], # till <bos>
                style_content_vectors = style_content_vectors,  # None,
                segments = segments[:, 0:init_context_size],# till <bos>
                eos_token_id = eos_token_id, # <eos>
                num_samples = num_samples, # 1
                length = generation_length,
                temperature = temparature,
                top_k = top_k,
                top_p = top_p,
                get_scores = True,
                interpolation = interpolation
            )

  return out, 0, scores

In [ ]:
def generate_batch(paraphraser, contexts, tokenizer):
  # global dense feature is none for every sample
  global_dense_features = [None for _ in contexts]

  examples = []

  for context, gdf in zip(contexts, global_dense_features):
    context_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(context))
    dd = {"sent1_tokens":context_ids, "sent2_tokens":context_ids}
    global_dense_vectors = np.zeros((1, 768), dtype=np.float32)

    dd_process = preprocess(dd, tokenizer, do_tokenize=False)
    dd_process["gdv"] = global_dense_vectors
    examples.append(dd_process)

  #-
  init_context_size = 1 + BASE_CONFIG["max_prefix_length"]

  output, _, scores = generate(
      paraphraser,
      sents = torch.tensor([inst["sentence"] for inst in examples]).to(device)
      segments = torch.tensor([inst["segment"] for inst in examples]).to(device),
      global_dense_vectors = torch.tensor([inst["gdv"] for inst in examples]).to(device),
      init_context_size = init_context_size
      eos_token_id = tokenizer.eos_token_id,
      get_scores = False,
      interpolation = None,
      top_p = top_p
  )

  all_ops = []
  for idx in range(len(output)):
    exmp = examples[idx]
    curr_out = output[idx, init_context_size:].tolist()

    if tokenizer.eos_token_id in curr_out:
      curr_out = curr_out[:curr_out.index(tokenizer.eos_token_id)]

    if upper_length.startswith("same"):
      extra = int(upper_length.split("_")[-1])
      curr_out = curr_out[:len(exmp["sent1_tokens"]) + extra]

    all_ops.append(
      tokenizer.decode(curr_out, clean_up_tokenization_spaces=True, skip_special_tokens=True)
            )

    return all_ops, scores

In [ ]:
# 2. Convert shakespeare sentences to their paraphrased version
for split in ["train", "test", "dev"]:
  file_dir = "" # that contains the sentences in shakesperean style
  label_dir = "" # that contains the style sentences like original, modern, romantic_peotry

  with open(file_dir, "r") as f:
    data = f.read().strip().split("\n")

  paraphrased_sents = []
  for i in tqdm.tqdm(range(0, len(data), batch_size), desc="minibatches of {} split done...".format(split)):
      generations, _ = generate_batch(paraphraser, data[i:i + batch_size], tokenizer)
      paraphrased_sents.extend(generations)

In [ ]:
# 3. Convert sentences to their bpe files
roberta = torch.hub.load('pytorch/fairseq', 'roberta.base')
bpe_outputs = [roberta.bpe.encode(x) for x in outputs]

In [ ]:
# save as train.paraphrase_250_input0.bpe

In [ ]:
# convert actual sents to .bpe file so that they can be directly use in inverse_paraphraser